In [ ]:
import pandas as pd
import numpy as np
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
import sys
sys.path.append('..')
from credentials import Credentials

In [ ]:
# Setting pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

In [ ]:
session.use_role("SYSADMIN")
session.use_database("STITCH")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("SALESFORCEFSL3")

# Indiviual Table 

In [ ]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEFSL3'
table_name:str = 'GROUP'

In [ ]:
table_view_mapping = session.sql(f"SELECT * FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_TABLE_VIEW_MAPPING WHERE NAME_SALESFORCE_ENVIRONMENT = '{schema_name}' AND NAME_SALESFORCE_TABLE_ORIGINAL = '{table_name}'".format(schema_name, table_name)).collect()
table_view_mapping_dic = [row.asDict() for row in table_view_mapping][0]
table_name = table_view_mapping_dic['NAME_SALESFORCE_TABLE_ORIGINAL']
table_for_desc_name = table_view_mapping_dic['NAME_SNOWFLAKE_RESERVED_WORD']
view_name = table_view_mapping_dic['NAME_SALESFORCE_VIEW_ALIAS']

In [ ]:
print(db_name, schema_name, table_name, table_for_desc_name, view_name)

In [ ]:
result_describe_table = [row.as_dict() for row in session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_for_desc_name};".format(db_name, schema_name, table_for_desc_name)).collect()]

In [ ]:
df_describe_table = pd.DataFrame(result_describe_table)
df_describe_table.drop(columns=['null?', 'default','primary key', 'unique key',
       'check', 'expression', 'comment', 'policy name'], inplace=True)

In [ ]:
df_describe_table.head(10)

## Mixing with Mapping table

In [ ]:
result_salesforce_mapping = [row.as_dict() for row in session.sql(f"SELECT NAME_SALESFORCE_ATTRIBUTE, NAME_ALIAS FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_MAPPING WHERE NAME_SALESFORCE_OBJECT = '{table_name}'".format()).collect()]
df_salesforce_mapping = pd.DataFrame(result_salesforce_mapping)

In [ ]:
df_salesforce_mapping

In [ ]:
if df_salesforce_mapping.empty:
	final_df = df_describe_table
	final_df['NAME_ALIAS'] = final_df['name']
else:
	final_df = pd.merge(df_describe_table, df_salesforce_mapping, left_on='name', right_on='NAME_SALESFORCE_ATTRIBUTE', how='left')
	final_df['NAME_ALIAS'] = final_df.apply(lambda x: x['name'] if pd.isna(x['NAME_ALIAS']) else x['NAME_ALIAS'], axis=1)


In [ ]:
final_df

In [ ]:
# a lambda function that creates a new column called 'name_new' if the type is like VARCHAR then add a "000" to the origial name else just add the original name
# df['name_new'] = df.apply(lambda x: x['name'] + '000' if isinstance(x['type'], str) and x['type'].find('VARCHAR') != -1 else x['name'], axis=1)

## Doing transformation

In [ ]:
def transform_name(name, type, name_alias):
    if isinstance(type, str) and 'VARCHAR' in type:
        #return name + '0007'
        return f"CAST(SUBSTR({name},1,10000) AS VARCHAR(10000)) AS {name_alias}".format(name, name_alias)
    else:
        return f"{name} AS {name_alias}".format(name, name_alias)

# apply the transform_name function to create a new column called 'name_new'
final_df['name_new'] = final_df.apply(lambda x: transform_name(x['name'], x['type'], x['NAME_ALIAS']), axis=1)

In [ ]:

#df[['name','type','name_new']].head(200)
column_list:list = final_df['name_new'].to_list()

In [ ]:
columns_str:str = ', '.join(column_list)

In [ ]:
columns_str

In [ ]:
# append the final result to something like "CREATE OR REPLACE VIEW {db_name}.{schema_name}}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"
final_query:str = f"CREATE OR REPLACE VIEW {db_name}.{schema_name}.{view_name} AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_for_desc_name};"

In [ ]:
final_query

In [ ]:
session.sql(final_query).collect()

# All tables

In [ ]:
database:str = 'STITCH'

for schema in ['SALESFORCEFSL3']:
    result = session.sql(f"SHOW TABLES IN STITCH.{schema}".format(schema)).collect()
    #[row.as_dict() for row in result]
    df = pd.DataFrame(result)
    tables:list = df['name'].to_list()
    for table in tables:
        if table not in ['_SDC_REJECTED']:
            print(table)
            create_view_query:str = f"CALL ACCOUNTADMIN_MGMT.UTILITIES.CREATE_DYNAMIC_SALESFORCE_VIEW('{database}', '{schema}', '{table}');".format(database, schema, table)
            session.sql(create_view_query).collect()


In [ ]:
session.close()